In [ ]:
repo_root = "/Users/miskodzamba/Dropbox/research/gits/spf/"
import sys

sys.path.append(repo_root)  # go to parent dir

from spf.rf import ULADetector
import numpy as np


"""
Ceiling / Roof

     x+ <-------- O
                  |
                  |
                  |
                  |
                  V y+

R(18,0)   R(17,0)
   \       /
    \     /
     \   /
       X
     /   \ 
    /     \ 
   /       \ 
R(17,1)   R(18,1)

Bearing 0 is straight towards floor

     o
     |
     |
     |
     V

Bearing 0+ is   

     o
   / |
 /   |
<    |
     V



Floor
"""

pos = ULADetector(
    sampling_frequency=None,
    n_elements=2,
    spacing=0.05075 * 1000,
    orientation=1.25 * np.pi,
).all_receiver_pos()
pos[1]

In [ ]:
pos = ULADetector(
    sampling_frequency=None,
    n_elements=2,
    spacing=0.05075 * 1000,
    orientation=0,
).all_receiver_pos()
pos[0]

In [ ]:
import matplotlib.pyplot as plt

rx_pos = np.array([1200, 1200])
tx_rads = np.linspace(0, 2 * np.pi, 9)  # rads from Y+ x=0, ro the right
tx_poss = 500 * np.vstack([np.sin(tx_rads), np.cos(tx_rads)]).T + rx_pos

for tx_idx in range(tx_rads.shape[0]):
    tx_pos = tx_poss[tx_idx]
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))
    for idx in [0, 1]:
        axs[idx].scatter(tx_pos[0], tx_pos[1], label="tx")
        axs[idx].scatter(rx_pos[0], rx_pos[1], label="rx")
        axs[idx].set_xlabel("X (mm)")
        axs[idx].set_ylabel("Y (mm)")
        axs[idx].set_xlim([0, 3000])
        axs[idx].set_ylim([0, 3000])
        axs[idx].legend()

    axs[0].set_title("Wall in canonical")
    axs[1].set_title("Wall as seen from desk")
    axs[1].invert_xaxis()
    axs[1].invert_yaxis()

    # compute the angle of the tx with respect to rx
    d = tx_pos - rx_pos
    fig.suptitle(
        f"measured_theta: {np.arctan2(d[0], d[1]):.2f}, actual: {tx_rads[tx_idx]:.2f}"
    )

In [ ]:
# phase conversion
"""
theta = angle from y+ axis between receiver array and target
        |      X            
        |theta/ 
        |    /
        |   / 
        |  /
x-------o---------x

d = distance between receivers
delta = difference in distance travelled by wave to receiver 1
cos(pi/2 - theta) = delta/d = sin(theta)

delta = sin(theta)

phase_diff = (delta / wavelength ) * 2 * pi = sin(theta) * d * 2 * pi / wavelength

theta = sin_inv(wavelength * phase_diff / (d*2*pi))

"""


"""
get_phase_diff defines angle(ant(0))-angle(ant(1))



"""


def phase_diff_to_theta(phase_diff, wavelength, distance_between_receivers):
    x = np.arcsin(wavelength * phase_diff / (distance_between_receivers * np.pi * 2))
    return x, np.pi - x